In [1]:
import numpy as np
import scanpy as sc
import os, sys

In [2]:
outdir = "pbmc_dists"
if not os.path.exists(outdir):
    os.makedirs(outdir)

## Load the count matrix

In [3]:
groups = ["Cortex", "PBMC"]
g = groups[1]

In [4]:
mtx = sc.read_mtx(g + "/matrix.mtx").transpose()

In [5]:
cells = []
with open(groups[1] + "/barcodes.tsv") as file:
    for line in file:
        cells.append(line.strip())

genes = []
with open(groups[1] + "/genes.tsv") as file:
    for line in file:
        genes.append(line.strip())

In [6]:
import pandas as pd

In [7]:
cell_df = pd.DataFrame(cells)
cell_df.index = cell_df[0]
cell_df = cell_df.drop(0, axis=1)
gene_df = pd.DataFrame(genes)
gene_df.index = gene_df[0]
gene_df = gene_df.drop(0, axis=1)
mtx.obs = cell_df
mtx.var = gene_df

In [8]:
mtx.obs["Experiment"] = [cell.split('.')[0] for cell in cells]
mtx.obs["Technology"] = [cell.split('.')[1] for cell in cells]

## Load marker gene data

In [9]:
import feather
markers = feather.read_dataframe(g + '/' + g + "_markers.feather")

/u/home/l/luke0321/nobackup-WGSPD/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [10]:
def remove_trailing_nums(cell_types):
    result = []
    for item in cell_types:
        for i in range(len(item) - 1, 0, -1):
            if not item[i].isdigit():
                result.append(item[:i + 1])
                break
    return result

In [11]:
markers["Cell type"] = remove_trailing_nums(markers["Cell type"])

In [12]:
up_genes = {}
down_genes = {}
for t in markers["Cell type"].unique():
    up_genes[t] = [item[:-1] for item in list(markers[markers["Cell type"] == t]["m"]) if item[-1] == '+']
    down_genes[t] = [item[:-1] for item in list(markers[markers["Cell type"] == t]["m"]) if item[-1] == '-']

## Load the optimal hyperparameters for clustering

In [13]:
params = pd.read_csv("params.txt", sep="\t", header=0)    

In [14]:
params

,Method,Replicate,#nearestneighbors,variable.gene,resolution,#PCs
0,Smart-seq2,PBMC1,5,False,1.5,20
1,CEL-Seq2,PBMC1,5,False,0.5,20
2,10x-Chromium-v2-A,PBMC1,30,False,1.5,30
3,10x-Chromium-v2-B,PBMC1,15,True,0.8,20
4,10x-Chromium-v3,PBMC1,30,True,0.8,30
5,Drop-seq,PBMC1,15,False,1.0,30
6,Seq-Well,PBMC1,15,True,1.5,30
7,inDrops,PBMC1,30,True,1.2,30
8,Smart-seq2,PBMC2,5,False,0.5,20
9,CEL-Seq2,PBMC2,5,True,0.5,20


In [15]:
mtx.obs["Technology"].unique()

array(['Smart-seq2', 'CEL-Seq2', '10x-Chromium-v2-A', '10x-Chromium-v2-B',
       '10x-Chromium-v3', 'Drop-seq', 'Seq-Well', 'inDrops',
       '10x-Chromium-v2'], dtype=object)

## Check the clustering and annotate each cluster with marker genes

In [16]:
from sklearn.metrics import roc_auc_score
from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist, euclidean
import matplotlib.pyplot as plt

In [17]:
def annotate_clusters(cells, up_genes, down_genes, groupby="louvain"):
    clusters = cells.obs["louvain"].unique()
    cell_types = [t for t in up_genes]
    cell_type_scores = np.zeros((cells.shape[0], len(cell_types)))
    observed_genes = list(cells.var.index)
    # Compute cell type collapsed gene scores for each cell
    up_down = [] # 1 for up, 0 for both, -1 for down
    for t in range(len(cell_types)):
        ups = list(set(observed_genes) & set(up_genes[cell_types[t]]))
        downs = list(set(observed_genes) & set(down_genes[cell_types[t]]))
        ups_index = []
        downs_index = []
        for item in ups:
            ups_index += [i for i in range(len(observed_genes)) if observed_genes[i] == item]
        for item in downs:
            downs_index += [i for i in range(len(observed_genes)) if observed_genes[i] == item]

        if len(ups) > 0 and len(downs) > 0:
            up_scores = np.sum(np.expm1(cells.X[:, ups_index]), axis=1) * 10e4 / np.sum(np.expm1(cells.X), axis=1)
            down_scores = np.sum(np.expm1(cells.X[:, downs_index]), axis=1) * 10e4 / np.sum(np.expm1(cells.X), axis=1)
            cell_type_scores[:, t] = np.log1p(up_scores).flatten() - np.log1p(down_scores).flatten()
            up_down.append(0)
        elif len(ups) > 0:
            scores = np.sum(np.expm1(cells.X[:, ups_index]), axis=1) * 10e4 / np.sum(np.expm1(cells.X), axis=1)
            cell_type_scores[:, t] = np.log1p(scores).flatten()
            up_down.append(1)
        else:
            scores = np.sum(np.expm1(cells.X[:, downs_index]), axis=1) * 10e4 / np.sum(np.expm1(cells.X), axis=1)
            cell_type_scores[:, t] = np.log1p(scores).flatten()
            up_down.append(-1)
    
    cluster_map = {}
    # Each cluster is assigned to the cell type with largest auc
    cluster_aucs = []
    for c in clusters:
        index = cells.obs["louvain"] == c
        index_not = cells.obs["louvain"] != c
        scores_c = cell_type_scores[index]
        scores_notc = cell_type_scores[index_not]
        aucs = np.zeros(len(cell_types))
        for t in range(len(cell_types)):
            if up_down[t] != -1:
                c_true = [1] * len(scores_c) + [0] * len(scores_notc)
            else:
                c_true = [0] * len(scores_c) + [1] * len(scores_notc)
            c_pred = np.append(scores_c[:, t], scores_notc[:, t])
            aucs[t] = roc_auc_score(c_true, c_pred)
        cluster_map[c] = cell_types[np.argmax(aucs)]
        cluster_aucs.append(np.max(aucs))
        
    return cluster_map, np.mean(cluster_aucs)

In [18]:
def cell_type_distances(cells, metric="euclidean"):
    cell_types = cells.obs["Cell type"].unique()
    cells_by_type = []
    for c in cell_types:
        cells_c = cells[cells.obs["Cell type"] == c]
        pcs_c = cells_c.obsm['X_pca']
        cells_by_type.append(pcs_c)
    distance_matrix = np.zeros((len(cell_types), len(cell_types)))
    for i in range(len(cell_types)):
        for j in range(i + 1, len(cell_types)):
            if metric == "eucentroid":
                distance_matrix[i][j] = euclidean(np.mean(cells_by_type[i], axis=0), np.mean(cells_by_type[j], axis=0))
            else:
                distance_matrix[i][j] = np.mean(cdist(cells_by_type[i], cells_by_type[j], metric=metric))
    return distance_matrix, cell_types

In [19]:
dists = ["euclidean", "cosine", "correlation", "cityblock", "eucentroid"]

In [20]:
def cluster_single_trial(mtx, experiment, technology, up_genes, down_genes, nn=10, variable_gene=False, reso=1.0, pc=30):
    ind = (mtx.obs["Technology"] == technology) & (mtx.obs["Experiment"] == experiment)
    if len(ind) == 0:
        return -1
    cells = mtx[ind].copy()
    # Annotate highly variable genes, this expects logarithmized data
    sc.pp.highly_variable_genes(cells)
    # Subset highly variable genes
    cells_highvar = cells[:, cells.var["highly_variable"] == True].copy()
    if variable_gene == True:
         # Perform pca
        sc.pp.pca(cells_highvar)
        # Compute k-NN graph
        sc.pp.neighbors(cells_highvar, n_neighbors=nn, n_pcs=pc)
        # Louvain algorithm for clustering
        sc.tl.louvain(cells_highvar, resolution=reso, random_state=0)
        cells.obs["louvain"] = cells_highvar.obs["louvain"]
    else:
        # Perform pca
        sc.pp.pca(cells)
        # Compute k-NN graph
        sc.pp.neighbors(cells, n_neighbors=nn, n_pcs=pc)
        # Louvain algorithm for clustering
        sc.tl.louvain(cells, resolution=reso, random_state=0)
    
    m, auc = annotate_clusters(cells, up_genes, down_genes)
    clusters = list(cells.obs["louvain"])
    cell_types = [m[c] for c in clusters]
    # t-SNE visualization
    if variable_gene == True:
        sc.pp.pca(cells)
    cells.obs["Cell type"] = cell_types
    sc.tl.tsne(cells, n_pcs=pc)
    plt.figure(figsize=(10, 10))
    sc.pl.tsne(cells, size=50, color="Cell type", title="%s, %s" % (experiment, technology), return_fig=True, show=False)
    lgd = plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')    
    plt.savefig(outdir + "/%s_%s.png" % (experiment, technology), bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.close()
    
    # Compute cell-type distances using a number of metrics
    for d in dists:
        m, c = cell_type_distances(cells, metric=d)
        # Write the distance matrix to a file
        with open(outdir + "/%s_%s_%s.txt" % (experiment, technology, d), "w+") as file:
            file.write('\t'.join(c) + '\n')
            for i in range(m.shape[0]):
                file.write('\t'.join([str(item) for item in m[i, :]]) + '\n')
    return

In [21]:
# Remove genes that are not expressed
sc.pp.filter_genes(mtx, min_counts=1)
# Normalize each cell by total gene counts
sc.pp.normalize_total(mtx)
# Log transformation
sc.pp.log1p(mtx)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [22]:
for exp in mtx.obs["Experiment"].unique():
    for tech in mtx.obs["Technology"].unique():
        params_row = params[params["Method"] == tech]
        params_row = params_row[params_row["Replicate"] == exp]
        if len(params_row) == 0:
            continue
        nn = int(params_row["#nearestneighbors"])
        vg = bool(params_row["variable.gene"].tolist()[0])
        reso = float(params_row["resolution"])
        pc = int(params_row["#PCs"])
        cluster_single_trial(mtx, exp, tech, up_genes, down_genes, nn=nn, variable_gene=vg, reso=reso, pc=pc)
        print("Finished %s, %s" % (exp, tech))

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'Experiment' as categorical
... storing 'Technology' as categorical
... storing 'Cell type' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Finished PBMC1, Smart-seq2


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'Experiment' as categorical
... storing 'Technology' as categorical
... storing 'Cell type' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Finished PBMC1, CEL-Seq2


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
... storing 'Experiment' as categorical
... storing 'Technology' as categorical
... storing 'Cell type' as categorical
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>